In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%writefile twitter_analysis_pagerank.py
import pyspark, time
import sys
from pyspark import SparkContext, SparkConf
import os

sc = pyspark.SparkContext()

ego_users = [20446839, 26961048, 7670202, 154701532, 258145892, 110755733, 39157827, 16824073, 77404713, 11928542, 13809612, 135605269,
252415267, 61086747, 250447400, 434433610, 27595997, 190696559, 314316607, 14045302, 239434259, 117800618, 91067832, 261973875,
374007416, 117671022, 18895362, 21028234, 38703025, 54460963, 44130462, 69049114, 54791004, 17797990, 33794839, 255790981, 111374622,
428333, 20436059, 107418464, 14371740, 122216533, 276128869, 14147754, 19976791, 90072587, 528575851, 14365883, 14791232, 263224927,
269559495, 18681592, 187744415, 151246522, 15070932, 30313925, 30207757, 18665800, 17403755, 28490110, 16183380, 17135931, 16613128,
47854857, 134943586, 1608991, 29034979, 18718580, 262310943, 9524062, 216843160, 21061498, 477094958, 59588845, 15477120, 13179562,
17922109, 18951737, 18135029, 83883736, 14917050, 22954430, 15859268, 12069912, 202346485, 61158706, 484991854, 33080832, 117809922,
16734718, 321652589, 278181157, 165786567, 17770689, 183786731, 87771546, 33868543, 86560711, 73024519, 14618160, 66804457, 21391704,
55928029, 86775971, 437300342, 19225156, 127912925, 54226675, 65357070, 242073660, 2589521, 22252971, 287519521, 18988524, 19948202,
61311054, 14827526, 15989218, 17866414, 303886358, 15001424, 54107218, 355823615, 96907622, 14984096, 301297627, 68179571, 1775731,
23469247, 18486243, 384761809, 82726142, 17045060, 360025190, 40777046, 155976326, 250340951, 15347317, 69548480, 15797184, 40813518,
120679977, 124296976, 48132655, 78944566, 71091272, 197903282, 13747362, 80479756, 158874150, 94147192, 188102842, 382110320, 14719129,
400867704, 19064433, 190138306, 17918629, 14630490, 100318079, 217796457, 232006900, 121202578, 47673661, 64619842, 345569115, 6297852,
364664695, 105398724, 42522911, 23742633, 9460682, 65185224, 326259198, 19110632, 42676302, 219365298, 15849055, 49253437, 16279105,
114115460, 17658786, 3253671, 306445007, 336463060, 123684709, 52772629, 45857296, 10985532, 50145896, 612473, 298038575, 623623,
22915745, 24117694, 356963, 87004822, 88639412, 113196940, 94915681, 58930815, 14077024, 240740059, 23759573, 266464616, 112628165,
229969377, 239141315, 15331855, 94480069, 14353392, 155689319, 19213083, 135895585, 18886852, 143172151, 21363077, 30393917, 1367531,
18068719, 14848513, 62985251, 14836915, 93006320, 115911638, 49414491, 232706326, 407769660, 170729553, 14555534, 21415375, 35926608,
73298877, 25755158, 191896931, 204646586, 745823, 2363991, 21077642, 230880104, 158414847, 15292446, 186909156, 199483163, 58641627,
25649234, 14840869, 12152402, 150402542, 66100530, 184391654, 151175266, 143661263, 291403100, 17111909, 16726878, 31153819, 37977732,
6650612, 45333725, 15507297, 15208246, 65360846, 33097148, 53685618, 311227912, 111045742, 69193714, 36836787, 778446, 52442002,
28205087, 215824411, 22745706, 742143, 253863144, 6408382, 344966590, 46160500, 2576401, 237049769, 18472735, 14313005, 20179628,
15947217, 68257605, 78138151, 24542441, 248224845, 2029971, 89826562, 15329577, 23889102, 116808228, 163374693, 16869298, 35012277,
38761811, 92715435, 191916463, 74264908, 5747502, 128135741, 51694885, 163491360, 37723441, 21132431, 90998968, 14528221, 256497288,
 176589903, 324201646, 78813, 14332288, 75343905, 22751884, 9855382, 289738351, 202114894, 228270980, 39322973, 18558612, 18630326,
 117029850, 144723379, 54331626, 21337501, 23334169, 14129601, 224581852, 532617990, 114885421, 61781462, 5471892, 35166642, 353767514,
 30824363, 45753733, 18496986, 36475508, 160237722, 11681802, 198327282, 430313102, 132519763, 78391198, 121685268, 40705032, 87798068,
 40022380, 48730516, 297985585, 89032548, 57828817, 18374320, 135397331, 18483115, 37170886, 13275962, 35012599, 18278524, 62609430,
  171202083, 17137927, 53235381, 494538543, 66185823, 58427526, 17767841, 107511013, 26234692, 276843589, 11784842, 397688944, 2097571,
  308220010, 18486881, 18846990, 153245741, 16027078, 15924858, 116388817, 175529494, 21339164, 6351572, 15706128, 159629146, 15164892,
  198628427, 163629705, 14755105, 49098552, 254883790, 7861312, 926981, 18996905, 333966373, 19283723, 31610343, 6018292, 19136295,
  72339761, 13811562, 113509970, 84473837, 31477674, 17723880, 90880254, 279854450, 75321229, 31464977, 67378554, 158419434, 80660928,
  493138720, 36629388, 14869783, 18270489, 3338681, 83048001, 16674149, 18338485, 124474025, 333881828, 238264600, 11348722, 85432934,
  296218728, 96483973, 227785045, 42098834, 46533358]

def get_sources_and_destinations(raw_edges_file):
  """ Modified version of 2(a). from lab 4, reads in text file and converts it into an RDD of format: (source, [destinations])"""
  raw_edges_file = sc.textFile(raw_edges_file)
  edges_rdd = raw_edges_file.map(lambda x: (int(x.split(',')[0].split('r=')[1].strip("'")), int(x.split(',')[1].split('d=')[1].split(')')[0].strip("'"))))
  graph_rdd = edges_rdd.groupByKey().map(lambda x: (x[0], list(x[1])))
  return graph_rdd

def get_col_trans_matrix(graph_rdd):
  def get_length(destinations):
    destinations = set(destinations)
    output = {}
    for item in destinations:
      output[item] = 1/len(destinations)
    return output
  col_matrix = graph_rdd.map(lambda x: (x[0], get_length(x[1])))
  return col_matrix

def col_to_row_matrix(col_trans_matrix):
    row_matrix = col_trans_matrix.flatMap(lambda column: ((row, (column[0], column[1][row])) for row in column[1])).groupByKey().sortByKey()
    return row_matrix

def row_multiply(row, R):
    result = 0
    for column, value in row:
      if column in R:
        result += value * R[column]
    return result

def produce_main_input(file):
  input = col_to_row_matrix(get_col_trans_matrix(get_sources_and_destinations(file)))
  return input.sample(False, 0.3, 999)

def page_rank_main(graph_rows, iterations=90, convergence_threshold=0.001):
    graph_rows.persist()
    N = graph_rows.count()
    damping_factor = 0.85
    R = graph_rows.map(lambda x: (x[0], 1/N)).collectAsMap()
    previous_R = R.copy()
    for t in range(iterations):
        vecR = sc.broadcast(R)
        row_results = graph_rows.map(lambda kv: (kv[0], row_multiply(kv[1], vecR.value)))
        R = row_results.reduceByKey(lambda a, b: damping_factor*a + (1 - damping_factor)/N + b).collectAsMap()
        # Check if values have converged
        delta = sum(abs(R[i] - previous_R[i]) for i in R.keys())
        if delta < convergence_threshold:
            break # Stops the loop, preventing unnecessary iterations
        previous_R = R.copy()
    # Sort by rank and return the top 10 results
    top_results = row_results
    graph_rows.unpersist()
    return top_results.filter(lambda x: x[0] in ego_users).sortBy(lambda x: -x[1]).take(10)

time_start = time.time()

page_rank = page_rank_main(produce_main_input(sys.argv[1]))
print(page_rank)

time_end = time.time()
print(f"elapsed time is {time_end-time_start}")

Overwriting twitter_analysis_pagerank.py


In [ ]:
USERNAME="lfoy"
%env REGION=australia-southeast1
%env ZONE=australia-southeast1-a
%env PROJECT=data301-2023-$USERNAME
%env CLUSTER=data301-2023-$USERNAME-twitter-analysis-cluster
%env BUCKET=data301-2023-$USERNAME-twitter-analysis-bucket

env: REGION=australia-southeast1
env: ZONE=australia-southeast1-a
env: PROJECT=data301-2023-lfoy
env: CLUSTER=data301-2023-lfoy-twitter-analysis-cluster
env: BUCKET=data301-2023-lfoy-twitter-analysis-bucket


In [ ]:
!python3 -m pip install google-cloud-dataproc[libcst]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=aEBvUmAvHZSHStlQPh34c4yvw5sSdY&prompt=consent&access_type=offline&code_challenge=Sm_Gxk18yTV0ln7MaAL9lH-3GHehf36h6osM8JuHbds&code_challenge_method=S256

Enter authorization code: 4/0AbUR2VNVLNGpqBwoZMSlXl-XPEpn7_bSsYno1_0c4DODKeZMhlbz48fK1EVUDhVDrn0UKw

You are now logged in as [lukafoy2@gmail.com].
Your current project is [data301-2023-lfoy].  You can change this setting by running:
  $ gcloud config set project PROJ

In [ ]:
!gcloud config set project $PROJECT


Updated property [core/project].


In [ ]:
!gcloud services enable dataproc.googleapis.com cloudresourcemanager.googleapis.com


Operation "operations/acat.p2-652813555918-176236de-fd79-4e9a-bb74-ea78d8930d3a" finished successfully.


In [ ]:
!gsutil mb -c regional -l $REGION -p $PROJECT gs://$BUCKET

Creating gs://data301-2023-lfoy-twitter-analysis-bucket/...
ServiceException: 409 A Cloud Storage bucket named 'data301-2023-lfoy-twitter-analysis-bucket' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [ ]:
!gcloud dataproc clusters create $CLUSTER --region=$REGION --bucket=$BUCKET --zone=$ZONE \
--master-machine-type=custom-1-6144 \
--image-version=1.5 --max-age=30m --single-node

Waiting on operation [projects/data301-2023-lfoy/regions/australia-southeast1/operations/7b78fad2-921b-305b-9d1f-a066699e0596].

Created [https://dataproc.googleapis.com/v1/projects/data301-2023-lfoy/regions/australia-southeast1/clusters/data301-2023-lfoy-twitter-analysis-cluster] Cluster placed in zone [australia-southeast1-a].


In [ ]:
!gcloud dataproc jobs submit pyspark --cluster=$CLUSTER --region=$REGION twitter_analysis_pagerank.py -- gs://$BUCKET/edges_rdd10.txt

Job [be9441e0512d4f87a1f1eb47850c81cb] submitted.
Waiting for job output...
23/06/01 10:10:16 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/06/01 10:10:16 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/06/01 10:10:16 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator
23/06/01 10:10:16 INFO org.spark_project.jetty.util.log: Logging initialized @5602ms to org.spark_project.jetty.util.log.Slf4jLog
23/06/01 10:10:16 INFO org.spark_project.jetty.server.Server: jetty-9.4.z-SNAPSHOT; built: unknown; git: unknown; jvm 1.8.0_362-b09
23/06/01 10:10:16 INFO org.spark_project.jetty.server.Server: Started @5936ms
23/06/01 10:10:17 INFO org.spark_project.jetty.server.AbstractConnector: Started ServerConnector@41b18748{HTTP/1.1, (http/1.1)}{0.0.0.0:39213}
23/06/01 10:10:19 INFO org.apache.hadoop.yarn.client.RMProxy: Connecting to ResourceManager at data301-2023-lfoy-twitter-analysis-cluster-m/10.152.0.27:8032
23/06/01 10:10:19 INFO org.apache.hado

In [ ]:
!gcloud dataproc clusters delete $CLUSTER --region=$REGION --quiet

Waiting on operation [projects/data301-2023-lfoy/regions/australia-southeast1/operations/dd979acd-7e03-3e9b-a1b9-b2b5e2943758].
Deleted [https://dataproc.googleapis.com/v1/projects/data301-2023-lfoy/regions/australia-southeast1/clusters/data301-2023-lfoy-twitter-analysis-cluster].
